##**A1.4 Selección de características**

***

**1.** Se importa la libreria pandas, se carga el archivo "A1.4 Vino Tinto.csv". Posterior a esto se imprimen las dimensiones del data frame, así como las primeras 5 filas del archivo.  

In [4]:
import pandas as pd
data=pd.read_csv('A1.4 Vino Tinto.csv')
print("Dimensiones del data frame:",data.shape)
data.head(5)

Dimensiones del data frame: (1599, 12)


,acidezFija,acidezVolatil,acidoCitrico,azucarResidual,cloruros,dioxidoAzufreLibre,dioxidoAzufreTotal,densidad,pH,sulfatos,alcohol,calidad
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


***

**2.** Se separa el data frame en datos de entrenamiento y de prueba para entrenar el modelo. En este caso se utiliza una proporción 80/20, siendo 80% de entrenamiento y 20% de prueba. Esta partición se hace de manera aleatoria y el final se confirma que ambas parten sumen el 100% de los datos.

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size = 0.8, random_state=42)
total=train.shape[0]+test.shape[0]

print("Observaciones datos de entrenamiento:",train.shape)
print("Observaciones datos de prueba:",test.shape)
print("Cantidad de observaciones totales:",total)

Observaciones datos de entrenamiento: (1279, 12)
Observaciones datos de prueba: (320, 12)
Cantidad de observaciones totales: 1599


***

**3.** Se importan las funciones *train_test_split*, *LinearRegression* y *SequentialFeatureSelector* cada una de sus respectivas librerías. 

Posterior a esto se implementa la metodología de selección de características hacia adelante. Para esto se separan las características y la variable objetivo. Luego, se define el modelo de regresión e implementa la selección hacia adelante con la ayuda de la función *SequentialFeatureSelector*.

Se seleccionan las características por este método y se imprimen tanto los índices como los nombres de estas.

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

x_train=train.iloc[:,:-1]
y_train=train.iloc[:,-1]
modelo=LinearRegression()

seleccion_adelante=SFS(modelo,k_features=(2,8),forward=True,scoring='r2',cv=10)
seleccion_adelante.fit(x_train,y_train)

caracteristicas_seleccionadas= list(x_train.columns[list(seleccion_adelante.k_feature_idx_)])

print("Índices de las características seleccionadas:", seleccion_adelante.k_feature_idx_)
print("Nombres de las características seleccionadas:", caracteristicas_seleccionadas)

Índices de las características seleccionadas: (1, 4, 5, 6, 8, 9, 10)
Nombres de las características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


***

**4.** Se entrena el modelo a partir de las características seleccionadas por la selección hacia adelante junto con la ayuda de la función *fit*. Después se realiza la predicción del modelo con la función *predict* y los datos de prueba. En ambos casos se utiliza solamente los datos de las variables de interés.

Para medir la capacidad de predicción del modelo mediante esta selección se utiliza la métrica de la R2 con la función *r2_score*.

In [13]:
from sklearn.metrics import r2_score

x_train_seleccion=x_train[caracteristicas_seleccionadas]
modelo.fit(x_train_seleccion,y_train)

x_test_seleccion=test[caracteristicas_seleccionadas]
y_test=test.iloc[:,-1]
y_prediccion=modelo.predict(x_test_seleccion)

R2=r2_score(y_test,y_prediccion)
print("R2 prueba (selección adelante):",R2)

R2 prueba (selección adelante): 0.40126288354402984


***

**5.** Una vez realizada la selección hacia adelante, ahora se implementa la selección hacia atrás a partir de las variables seleccionadas por el método de selección hacia adelante.

Se realizan pasos muy similares a la parte 3, sin embargo en este caso la mayor diferencia es que ahora *forward=False*, esto para que vaya de atrás hacia adelante. Así mismo, se específica una menor cantidad de variables posibles, en este caso de 2-5.

Por último, se imprimen los índices y los nombres de las características seleccionadas por este método.

In [16]:
seleccion_atras=SFS(modelo,k_features=(2,5),forward=False,scoring='r2',cv=10)
seleccion_atras.fit(x_train_seleccion,y_train)

caracteristicas_seleccionadas_atras=list(x_train_seleccion.columns[list(seleccion_atras.k_feature_idx_)])

print("Índices de las características seleccionadas:", seleccion_atras.k_feature_idx_)
print("Nombres de las características seleccionadas:", caracteristicas_seleccionadas_atras)

Índices de las características seleccionadas: (0, 1, 3, 5, 6)
Nombres de las características seleccionadas: ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


***

**6.** Teniendo la selección de características por el método hacia atrás se repiten los pasos similar a la parte 4. En esta parte se asegura que el modelo contenga solamente las variables seleccionadas en el parte 5. 

Finalmente, luego de realizar el ajuste y la predicción, se calcula la R2 del modelo por el método hacia atrás. Al tener ambos valores de la R2 se comparan para evaluar cual de los dos modelos sería el mejor.

In [19]:
x_train_seleccion_atras=x_train[caracteristicas_seleccionadas_atras]
modelo.fit(x_train_seleccion_atras,y_train)

x_test_seleccion_atras=test[caracteristicas_seleccionadas_atras]
y_prediccion_atras=modelo.predict(x_test_seleccion_atras)

R2_atras=r2_score(y_test,y_prediccion_atras)
print("R2 prueba (selección atrás):",R2_atras)

if R2 > R2_atras:
    print("El modelo con selección adelante tiene mejor desempeño con un R2 de", R2, "comparado con", R2_atras, "del modelo con selección atrás.")
elif R2_atras > R2:
    print("El modelo con selección atrás tiene mejor desempeño con un R2 de", R2_atras, "comparado con", R2, "del modelo con selección adelante.")
else:
    print("Ambos modelos tienen el mismo desempeño con un R2 de", R2)


R2 prueba (selección atrás): 0.3958889666765406
El modelo con selección adelante tiene mejor desempeño con un R2 de 0.40126288354402984 comparado con 0.3958889666765406 del modelo con selección atrás.


***

**Conclusión:** Con base a la métrica calculada de la R2, la cual mide qué tan bien el modelo puede explicar la variabilidad de la variable dependiente (𝑦), el modelo con el método de selección hacia adelante se podría decir que es mejor que el de selección hacia atrás puesto que un valor más cerca a 1 indica que el modelo explica una gran parte de la variabilidad de los datos. Es decir, con el primer modelo podemos decir que las predicciones se ajustan mejor a los valores reales.

Sin embargo, es importante mencionar que hay que tener en cuenta la parte del sobreajuste ya que un R2 muy alto en entrenamiento pero bajo en prueba indica que el modelo se ajustó mucho a los datos de entrenamiento y no generaliza bien. Aunado a esto, no siempre un R2 mayor implica un mejor modelo si el número de variables aumenta demasiado. Un mayor R2 es deseable, pero debe ser analizado en conjunto con otras métricas.

***

**Referencias bibliográficas**
 - Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Wine Quality [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C56S3T.